<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="img/cover-small.jpg" />

Dieses Notizbuch enthält einen angepassten Auszug aus der [Whirlwind Tour of Python](http://www.oreilly.com/programming/free/a-whirlwind-tour-of-python.csp) von Jake VanderPlas; Der Inhalt ist auf [GitHub](https://github.com/jakevdp/WhirlwindTourOfPython) verfügbar.

Text und Code werden unter der [CC0](https://github.com/jakevdp/WhirlwindTourOfPython/blob/master/LICENSE)- Lizenz veröffentlicht; Das Begleitprojekt, das [Python Data Science Handbook](https://github.com/jakevdp/PythonDataScienceHandbook) wird sehr empfohlen.


# Fehler und Ausnahmen (Errors and Exceptions)

Unabhängig von unseren Fähigkeiten als Programmierer:innen werden wir früher oder später einen Programmierfehler machen.

Wir unterscheiden grundsätzlich drei Varianten:

- *Syntaxfehler:* Fehler, bei denen der Code kein gültiger Python-Code ist (im Allgemeinen leicht zu beheben)
- *Laufzeitfehler:* Fehler, bei denen syntaktisch gültiger Code nicht ausgeführt werden kann, vielleicht aufgrund ungültiger Benutzereingaben (manchmal leicht zu beheben)
- *Semantische Fehler:* Fehler in der Logik: Der Code wird problemlos ausgeführt, aber das Ergebnis ist nicht das, was Sie erwarten (oft sehr schwer aufzuspüren und zu beheben)

Wir werden uns darauf konzentrieren, wie man mit *Laufzeitfehlern* sauber umgeht. Wie wir sehen werden, behandelt Python Laufzeitfehler über das integrierte Framework zur *Ausnahmebehandlung*.

## Laufzeitfehler

Alle, die schon einmal in Python programmiert haben, sind wahrscheinlich schon auf Laufzeitfehler gestoßen. Sie können auf verschiedene Arten auftreten.

Zum Beispiel, wenn wir versuchen, eine undefinierte Variable zu referenzieren:

In [1]:
print(Q)

NameError: name 'Q' is not defined

Oder wenn wir eine Operation ausprobieren, die nicht definiert ist:

In [2]:
1 + 'abc'

'1abc'

Oder wir versuchen, ein mathematisch unbestimmtes Ergebnis zu berechnen:

In [5]:
2 / 0

ZeroDivisionError: division by zero

Oder vielleicht versuchen wir, auf ein Sequenzelement zuzugreifen, das nicht existiert:

In [6]:
L = [1, 2, 3]
L[1000]

IndexError: list index out of range

Wir können erkennen, dass Python in jedem Fall so freundlich ist, nicht einfach anzuzeigen, dass ein Fehler aufgetreten ist. Wir erhalten eine *aussagekräftige* Fehlermeldung, die Informationen darüber enthält, was genau schiefgelaufen ist, zusammen mit der exakten Codezeile, in der der Fehler aufgetreten ist.

Der Zugriff auf aussagekräftige Fehlermeldungen wie diese ist ungemein nützlich, wenn man versucht, die Ursache von Problemen in seinem Code zu finden.

## Ausnahmen fangen: ``try`` und ``except``
Das wichtigste Werkzeug, das Python uns für die Behandlung von Laufzeitausnahmen zur Verfügung stellt, ist die 

 The main tool Python gives you for handling runtime exceptions is the ``try``...``except`` Klausel.

Die Grundstruktur dabei sieht so aus:

In [3]:
try:
    print("this gets executed first")
except:
    print("this gets executed only if there is an error")

this gets executed first


Hier wird der zweite Block nicht ausgeführt: Das liegt daran, dass der erste Block keinen Fehler zurückgegeben hat.
Wir wollen nun eine problematische Anweisung in den ``try``-Block einfügen und sehen, was passiert:

In [4]:
try:
    print("let's try something:")
    x = 1 / 0 # ZeroDivisionError
except:
    print("something bad happened!")

let's try something:
something bad happened!


Wir sehen hier, dass der Fehler, der in der ``try``-Anweisung ausgelöst wurde (in diesem Fall ein ``ZeroDivisionError``), abgefangen und die ``except``-Anweisung ausgeführt wurde.

Eine häufige Anwendung ist die Überprüfung von Benutzereingaben innerhalb einer Funktion oder eines anderen Codeteils. Wir könnten zum Beispiel eine Funktion erschaffen, die eine Null-Division abfängt und einen anderen Wert zurückgibt, vielleicht eine SEHR große Zahl wie $10^{100}$:

In [5]:
def safe_divide(a, b):
    try:
        return a / b
    except:
        return 1E100

In [10]:
safe_divide(1, 2)

0.5

In [6]:
safe_divide(2, 0)

1e+100

Allerdings gibt es ein kleines Problem mit diesem Code: Was passiert, wenn eine andere Art von Ausnahme auftritt? Dies ist zum Beispiel wahrscheinlich nicht das, was wir beabsichtigt haben:

In [7]:
safe_divide (1, '2') # Zeichenkette als Divisor

1e+100

Dividing an integer and a string raises a ``TypeError``, which our over-zealous code caught and assumed was a ``ZeroDivisionError``!
For this reason, it's nearly always a better idea to catch exceptions *explicitly*:

In [8]:
def safe_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return 1E100

In [9]:
safe_divide(1, 0)

1e+100

In [10]:
safe_divide(1, '2')

TypeError: unsupported operand type(s) for /: 'int' and 'str'

Wir fangen jetzt nur noch Null-Divisions-Fehler ab und reichen alle anderen Fehler unverändert weiter.

## Ausnahmen auslösen: ``raise``
Wir haben gesehen, wie wertvoll es ist, informative Fehlermeldungen zu erhalten, wenn man eigenen oder fremden Code aufruft.
Genauso wertvoll ist es, informative Fehlermeldungen in dem von uns geschriebenen Code zu erzeugen, so dass die Benutzer unseres Codes (einschließlich uns selbst!) herausfinden können, was die Fehler verursacht hat.

Wir lösen unsere eigenen Ausnahmen mit der Anweisung ``raise`` aus. Zum Beispiel:

In [11]:
raise RuntimeError("my error message")

RuntimeError: my error message

Als Beispiel dafür, wo dies nützlich sein könnte, kehren wir zu unserer ``fibonacci``-Funktion zurück, die wir zuvor definiert haben:

In [12]:
def fibonacci(N):
    L = []
    a, b = 0, 1
    while len(L) < N:
        a, b = b, a + b
        L.append(a)
    return L

Ein mögliches Problem ist, dass der übergebene Eingabewert negativ sein könnte.
Dies wird derzeit keinen Fehler in unserer Funktion auslösen, aber wir sollten den Benutzer wissen lassen, dass ein negatives ``N`` nicht unterstützt wird.
Fehler, die auf ungültige Parameterwerte zurückzuführen sind, führen gemäß der Konvention zu einem ``ValueError``:

In [13]:
def fibonacci(N):
    if N < 0:
        raise ValueError(f"N was {N}, but must be non-negative!")
    L = []
    a, b = 0, 1
    while len(L) < N:
        a, b = b, a + b
        L.append(a)
    return L

In [14]:
fibonacci(10)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [15]:
fibonacci(-10)

ValueError: N was -10, but must be non-negative!

Jetzt wissen die alle, die unseren Code nutzen, genau, warum die Eingabe ungültig ist, und können sogar einen ``try``...``except``-Block verwenden, um das Problem zu lösen!

In [16]:
N = -10
try:
    print("trying this...")
    print(fibonacci(N))
except ValueError: # NICHT optimal! (siehe unten)
    print("Bad value: need to do something else")

trying this...
Bad value: need to do something else


## Wir tauchen tiefer in die Ausnahmen ein...

Wir wollen uns noch einige Konzepte anschauen, die uns in Beispielcode begegnen könnten, ohne dabei auf jedes letzte Detail einzugehen. Wir machen uns mit der Syntax vertraut, wer möchte kann die Konzepte selbst weiter erkunden.

### Zugriff auf die Fehlermeldung

Manchmal möchte man in einer ``try``...``except``-Anweisung mit der Fehlermeldung selbst arbeiten. Dies kann mit dem ``as``-Schlüsselwort erreicht werden:

In [17]:
try:
    x = 1 / 0
except ZeroDivisionError as err:
    print("Error class is:  ", type(err))
    print("Error message is:", err)

Error class is:   <class 'ZeroDivisionError'>
Error message is: division by zero


Mit diesem Muster können wir die Behandlung von Ausnahmen in unseren Funktionen weiter anpassen.

### Definition von benutzerdefinierten Ausnahmen

Zusätzlich zu den eingebauten Ausnahmen können wir unsere eigenen Ausnahmen durch *Vererbungsbeziehungen* definieren.
Wenn wir zum Beispiel eine spezielle Art von ``ValueError`` haben wollen, können wir dies tun:

In [23]:
class MySpecialError(ValueError):
    pass

raise MySpecialError("here's the message")

MySpecialError: here's the message

Dies würde es uns ermöglichen, einen ``try``...``except``-Block zu verwenden, der nur diese Art von Fehler abfängt:

In [24]:
try:
    print("do something")
    raise MySpecialError("[informative error message here]")
except MySpecialError:
    print("do something else")

do something
do something else


Dies könnte sich als nützlich erweisen, wenn wir mehr eigenen Code entwickeln.

## ``try``...``except``...``else``...``finally``
Zusätzlich zu den Schlüsselwörtern ``try`` und ``except`` können wir die Schlüsselwörter ``else`` und ``finally`` verwenden, um die Behandlung von Ausnahmen in unserem Code weiter zu optimieren.
Die grundlegende Struktur ist die folgende:

In [25]:
try:
    print("try something here")
except:
    print("this happens only if it fails")
else:
    print("this happens only if it succeeds")
finally:
    print("this happens no matter what")

try something here
this happens only if it succeeds
this happens no matter what


Der Nutzen von ``else`` ist hier klar, aber was ist der Sinn von ``finally``?
Die ``finally``-Klausel wird wirklich *IMMER* ausgeführt: Normalerweise wird sie verwendet, um eine Aufräumajktionen nach Abschluss einer Operation durchzuführen.